In [ ]:
import pandas as pd
import numpy as np

def check_apt_grade(df, name_col='아파트명', grade_col='apt_grade', brand_col_candidates=('apt_brand','apt_brand_key')):
    print("=== 기본 정보 ===")
    print(f"rows: {len(df):,}, columns: {len(df.columns)}")
    assert grade_col in df.columns, f"컬럼 '{grade_col}' 이(가) 없습니다."
    assert name_col in df.columns, f"컬럼 '{name_col}' 이(가) 없습니다."

    # 타입/결측/범위 체크
    print("\n=== 타입/결측/범위 체크 ===")
    print(df[grade_col].describe(include='all'))
    na_cnt = df[grade_col].isna().sum()
    if na_cnt > 0:
        print(f"[경고] {grade_col} 결측치: {na_cnt:,}개")
    # 숫자 변환 시도
    try:
        df[grade_col] = pd.to_numeric(df[grade_col], errors='coerce')
    except Exception as e:
        print(f"[참고] 숫자 변환 실패: {e}")
    # 범위 점검
    invalid_mask = ~df[grade_col].isin([1,2,3,4,5])
    invalid_cnt = invalid_mask.sum()
    if invalid_cnt > 0:
        print(f"[경고] 1~5 범위를 벗어나는 값: {invalid_cnt:,}개 (상위 10개)\n", df.loc[invalid_mask, grade_col].head(10))
    else:
        print("범위 OK: apt_grade ∈ {1,2,3,4,5}")

    # 분포표
    print("\n=== 등급 분포 (건수/비율) ===")
    vc = df[grade_col].value_counts(dropna=False).sort_index()
    ratio = (vc / len(df) * 100).round(2)
    dist = pd.DataFrame({'count': vc, 'ratio_%': ratio})
    print(dist)

    # 브랜드 교차표 (있을 때)
    brand_col = None
    for c in brand_col_candidates:
        if c in df.columns:
            brand_col = c
            break
    if brand_col:
        print(f"\n=== 교차표: {brand_col} × {grade_col} (상위 브랜드 20개) ===")
        ct = pd.crosstab(df[brand_col], df[grade_col]).sort_values(by=[5,4,3,2,1], ascending=False, axis=0, errors='ignore')
        print(ct.head(20))
        # 매칭 실패(기타) 비율
        if '기타' in ct.index:
            etc_ratio = ct.loc['기타'].sum() / len(df) * 100
            print(f"\n'기타' 비율: {etc_ratio:.2f}%")
    else:
        print("\n브랜드 컬럼이 없어 교차표는 생략합니다. (옵션: 'apt_brand' 또는 'apt_brand_key')")

    # 고가 등급 샘플 출력
    print("\n=== 상위 등급 샘플 (4·5등급) ===")
    high = df[df[grade_col].isin([4,5])][[name_col, grade_col]].head(20)
    if len(high):
        print(high)
    else:
        print("4·5등급 샘플이 없습니다.")

    # 저/중/고 3단계로 단순화
    print("\n=== 3단계 bin (저가=1, 중가=2, 고가=3) 분포 ===")
    # 저가(1) / 중가(2·3) / 고가(4·5)
    bins_map = {1:1, 2:2, 3:2, 4:3, 5:3}
    df['apt_grade_bin'] = df[grade_col].map(bins_map).astype('Int64')
    vc_bin = df['apt_grade_bin'].value_counts().sort_index()
    ratio_bin = (vc_bin / len(df) * 100).round(2)
    dist_bin = pd.DataFrame({'count': vc_bin, 'ratio_%': ratio_bin})
    print(dist_bin)

    # (옵션) 고가 비중 과하지 않나 빠르게 체크
    if dist_bin.loc[3, 'ratio_%'] < 1:
        print("\n[참고] 고가(3) 비중이 1% 미만입니다. 필요 시 4·5를 묶은 의도대로 잘 적용된 것입니다.")
    return dist, dist_bin

# 실행
_ = check_apt_grade(train_main)  # df 이름/컬럼명이 다르면 인자 바꿔 호출

In [ ]:
# feature  importance
# 2            전용면적         0.420294
# 15            계약년          0.189463
# 20            시군구          0.124306
# 5            건축년도         0.077257
# 38              구            0.061788
# 0              본번           0.018096
# 29         k-난방방식         0.017344
# 39              동            0.014076
# 23            도로명          0.012125
# 21             번지           0.011913
# 1              부번           0.008706
# 12           주차대수         0.006107
# 22           아파트명         0.005086
# 4               층            0.004151
# 13            좌표X           0.004077
# 19          중대형비율        0.003310
# 14            좌표Y           0.003118
# 8           k-연면적          0.002964
# 31  k-사용검사일-사용승인일    0.002362
# 26   k-세대타입(분양형태)    0.001899